# Two Newsvendor Problem

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pedronahum/stochastic-optimization/blob/master/notebooks/two_newsvendor.ipynb)

In [ ]:
# Install JAX and dependencies
!pip install -q jax jaxlib jaxtyping chex numpy matplotlib

# Clone repository (force fresh clone for latest code)
import os
import shutil

if os.path.exists('stochastic-optimization'):
    shutil.rmtree('stochastic-optimization')

!git clone https://github.com/pedronahum/stochastic-optimization.git
os.chdir('stochastic-optimization')

# Clear Python import cache
import sys
for key in list(sys.modules.keys()):
    if key.startswith('problems'):
        del sys.modules[key]

print('✓ Setup complete!')

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

# Import problem components
from problems.two_newsvendor import (
    TwoNewsvendorConfig,
    TwoNewsvendorFieldModel,
    NewsvendorFieldPolicy
)

print('✓ Imports successful')
print(f'JAX version: {jax.__version__}')
print(f'JAX backend: {jax.default_backend()}')

In [ ]:
# Create model configuration
config = TwoNewsvendorConfig(
    demand_lower=0.0,
    demand_upper=100.0,
    overage_cost_field=1.0,
    underage_cost_field=9.0
)
model = TwoNewsvendorFieldModel(config)

# Create policy
policy = NewsvendorFieldPolicy(model)

key = jax.random.PRNGKey(42)
state = model.init_state(key)

print('✓ Two newsvendor model ready')
print('✓ Newsvendor policy created')

In [ ]:
# Demonstrate model and policy initialization
key = jax.random.PRNGKey(42)
state = model.init_state(key)

print(f'✓ Model initialized')
print(f'  State shape: {state.shape}')
print(f'  Initial state: {state}')

# Get a single decision
key, subkey = jax.random.split(key)
decision = policy(None, state, subkey)

print(f'\n✓ Policy works')
print(f'  Decision (order quantity): {decision[0]:.2f}')
print(f'\nNote: This is a multi-agent problem. For full coordination,')
print(f'see the test file: tests/test_two_newsvendor.py')